In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def gen_data(n, mu, sigma, p):
    labels = torch.bernoulli(torch.Tensor(n, 1).fill_(p))
    clusterId = torch.bernoulli(torch.Tensor(n, 1).fill_(0.75)).long()
    ones = torch.normal(mu*(2*clusterId-1), sigma)
    zeros = torch.normal(-mu*(2*clusterId-1), sigma)
    data = torch.where(labels==1, ones, zeros)
    return data, labels

In [24]:
def grad_by_data_on_model(model, xs):
    model.zero_grad()
    probPred = torch.sigmoid(model(xs))
    labelPred = (probPred > 0.5).float().detach()
    loss = F.binary_cross_entropy(probPred, labelPred)
    loss.backward()
    return model[0].weight.grad.data.clone(), model[0].bias.grad.data.clone()

In [46]:
n = 5000
d = 1
mu = 1.
sigma = 1.
pSource = 0.5
pTarget = .95

In [47]:
x, y = gen_data(1000, mu, sigma, pTarget)

In [48]:
model = nn.Sequential(nn.Linear(1, 1))
model[0].weight.data = torch.tensor([[1.]]).float()
model[0].bias.data = -torch.tensor(torch.sum(torch.mean(x, dim=0))).float()

/var/folders/_x/wtryhxsj37vd8b_d3pb8klt40000gn/T/ipykernel_1673/1133744714.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model[0].bias.data = -torch.tensor(torch.sum(torch.mean(x, dim=0))).float()


In [49]:
wGradPos, bGradPos = grad_by_data_on_model(model, x[x>0, None])
wGradNeg, bGradNeg = grad_by_data_on_model(model, x[x<0, None])

In [50]:
bGradNeg + bGradPos, wGradNeg + wGradPos

(tensor(0.0682), tensor([[-0.3802]]))